In [2]:
#Caculate PAF Function
def PAF(area_name,value):
    #Take the temperature of the highest frequency as the baseline statistics
    df_zone_value = sa[str(area_name)]
    area_code = ((df_zone_value-df_zone_value.min()).astype(int)).value_counts()
    area_code = pd.concat([area_code,(area_code/area_code.sum()).apply('{:.2%}'.format)], axis=1)
    area_code.index = area_code.index + df_zone_value.min()
    area_code.columns = ['Counts','Percentage']
    area_code = area_code.sort_index(ascending=False)
    area_code = area_code.loc[area_code.index >= (result_step2[str(area_name)][value]).max()]
    area_code = area_code.sort_index()
    area_code = area_code.drop(columns=['Counts'])
    area_code = area_code.reset_index()
    area_code_min = area_code['index'].min()
    area_code_min_max = area_code['index'].max()
    index = []
    percentage = [0 for x in range(int(area_code_min_max-area_code_min+1))]
    for i in range(int(area_code_min_max-area_code_min+1)):
        index.append(area_code_min)
        area_code_min += 1
    area_code_full = pd.DataFrame({'index':index,'Percentage':percentage})
    area_code = pd.merge(area_code, area_code_full,how='right' ,on='index' )
    area_code = area_code.sort_values(by='index').drop(columns=['Percentage_y']).reset_index(drop=True)
    area_code = area_code.fillna(value='0%')
    #PAF official
    n = 1
    PAF_numerator = []
    for index, row in area_code.iterrows():
        RR = math.pow(math.e,math.log(RR_dict[area_name]) * n)
# Sensitivity analysis: 'Non-lineaer functions'
#        RR = math.pow(math.e,(math.log(RR_dict[area_name]) + 1) * math.log(RR_dict[area_name]) * n)
#        RR = math.pow(math.e,(math.log(RR_dict[area_name]) * math.log(RR_dict[area_name]) + math.log(RR_dict[area_name]) + 1) * math.log(RR_dict[area_name]) * n)
        # Calculate PAF's numerator 
        numerator = (float(row[1].strip('%'))/100 * (RR-1))
        PAF_numerator.append(numerator)
        n += 1
    # Calculate PAF
    PAF = sum(PAF_numerator) / (sum(PAF_numerator)+1) * 100
    return PAF

In [3]:
import pandas as pd
import os
import glob
import math
import warnings
warnings.filterwarnings('ignore')#Hide warnings to make the interface look better
#(A) Load file
# Step 1 data - daily mean temperature across 2,310 statistical areas (level 2) in Australia
#get the path
path = os.getcwd()
#Get all of files name in the folder
filenames = glob.glob(path+"/Mean_*.csv")
#Loop files data and caculate PAF,then save to CSV, like: 'Mean_RCP85_2036_65PAF.csv'
for filename in filenames:
    df = pd.read_csv(filename)
# Sensitivity analysis: alternative 'TMREDs'
    df_step4 = pd.read_excel('MFT_Basline.xlsx')
    df_step4.set_index(['Indicators'],inplace=True)
# Sensitivity analyais: alternative 'Exposure-Response Functions'
    df_RR = pd.read_csv('MH_RR.csv')
    #Get RR Value
    df_RR= df_RR.set_index(['ASGS_2016']).T.iloc[-1:].reset_index(drop=True)
    df_RR = df_RR.loc[:,(df_RR!=0).any(axis=0)].T.reset_index()
    RR_dict = dict(zip(df_RR['ASGS_2016'],df_RR[0]))
    #Get the tmp area code
    list_RR = df_RR['ASGS_2016'].tolist()
    #Get all area zone code
    df_list = list(map(int,list(df.columns)[1:]))
    list_zone = [column for column in df_list if column in list_RR]
    for x in list(RR_dict.keys()-list_zone):
        del RR_dict[x]
    df_PAF_MFT = pd.DataFrame(columns=([x for x in list_zone]))
    n = 0
    while ((df.index.max()+1) -n) != 0:
        sa = df.iloc[n:n+365]
        year = sa.reset_index(drop=True).iloc[:, 0][0][0:5]
        sa = sa.drop(columns=['Unnamed: 0'])
        Climate_Zone_MFT = df_step4.iloc[0:1]
        Climate_Zone_MFT.index = ['MFT']
        result_step2 = pd.concat([sa.describe(), Climate_Zone_MFT])
        n += 365
        print(year)
        df_PAF = pd.DataFrame([[PAF(x,'MFT') for x in list_zone]])
        df_PAF.columns = [x for x in list_zone]
        df_PAF.index = [filename[-22:-4] + year + 'PAF']
        df_PAF_MFT = pd.concat([df_PAF_MFT,df_PAF])
    df_PAF_MFT.to_csv(filename[-22:-4]+'MHPAFRR.csv')

X2003
X2004
X2005
X2006
X2007
X2008
X2009
X2010
X2011
X2012
X2013
X2014
X2015
X2016
X2017
X2018
